# 自动微分机制
torch中的核心概念是torch.tensor, 即张量。

张量的重要性质是 .requires_grad, 如果设置它的属性 .requires_grad 为 True，那么它将会追踪对于该张量的所有操作。当完成计算后可以通过调用 .backward()，来自动计算所有的梯度。这个张量的所有梯度将会自动累加到.grad属性。  对于自己定义的张量，默认.requires_grad=False, 通过torch神经网络模型生成的张量（包括网络的可训练参数）默认为True。

除此之外，还有两个性质：.grad 和 .grad_fn

(1).grad是一个指向该张量创建过程的函数的引用，它指示了这个张量是通过某种操作（例如加法、矩阵乘法等）生成的。它保存着该张量的计算历史，因此可以追踪它在计算图中的前向传播过程。如果这个张量是由用户直接创建的（例如通过 torch.tensor() 创建的常量张量），它的 grad_fn 为 None。

(2).grad是一个张量的属性，用来存储该张量的梯度。梯度是通过反向传播计算得到的，它代表了该张量相对于某个标量值（通常是损失函数）的导数。在反向传播（backward()）后，grad 属性会存储计算得到的梯度值。通常，你会通过 tensor.grad 来访问这个梯度。如果一个张量没有被标记为需要梯度（即 requires_grad=False），或者没有进行反向传播，grad 就会是 None。

当然，可以阻止计算被追踪。除了将.requries_grad设置为False外，可以通过将代码块包装在 with torch.no_grad(): 中，来阻止 autograd 跟踪设置了.requires_grad=True的张量的历史记录。

In [1]:
import os
import numpy as np
import torch
import torch.nn as nn

In [49]:
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 1))

In [50]:
for name, param in net.named_parameters():
    print(f"Layer: {name}, Shape: {param.shape}, {param.requires_grad}, {param.grad_fn}, {param.grad}")

Layer: 0.weight, Shape: torch.Size([256, 20]), True, None, None
Layer: 0.bias, Shape: torch.Size([256]), True, None, None
Layer: 2.weight, Shape: torch.Size([1, 256]), True, None, None
Layer: 2.bias, Shape: torch.Size([1]), True, None, None


In [54]:
# 生成随机输入（batch_size=10, 输入特征维度=20）
input_data = torch.randn(10, 20)  # 10个样本，每个样本20维

# 前向传播
output = net(input_data)  # 计算输出

# 定义损失函数（假设是回归任务，使用均方误差损失）
loss_fn = nn.MSELoss()
target = torch.randn(10, 1)  # 随机生成目标值
loss = loss_fn(output, target)  # 计算损失

# 反向传播
loss.backward()  # 计算梯度

In [56]:
# 反向传播后.grad_fn和.grad属性不再是None
for name, param in net.named_parameters():
    print(f"Layer: {name}, Shape: {param.shape}, {param.requires_grad}, {param.grad_fn}, {param.grad}")

Layer: 0.weight, Shape: torch.Size([256, 20]), True, None, tensor([[-0.0063, -0.0110,  0.0008,  ...,  0.0019, -0.0017,  0.0031],
        [-0.0037, -0.0250,  0.0034,  ..., -0.0187,  0.0047, -0.0020],
        [ 0.0147,  0.0230,  0.0092,  ..., -0.0045, -0.0030, -0.0060],
        ...,
        [-0.0016, -0.0247, -0.0006,  ..., -0.0378, -0.0037,  0.0063],
        [ 0.0052,  0.0164, -0.0107,  ...,  0.0380,  0.0040, -0.0056],
        [-0.0024, -0.0184, -0.0044,  ...,  0.0008,  0.0002, -0.0038]])
Layer: 0.bias, Shape: torch.Size([256]), True, None, tensor([ 2.6050e-03,  1.7577e-02,  2.0176e-03,  0.0000e+00, -4.8657e-03,
         4.3071e-03,  1.5231e-02, -3.4564e-02, -1.9429e-03,  1.9044e-02,
        -5.0433e-05, -1.4930e-02,  2.1513e-02,  1.9944e-03, -1.9099e-02,
        -9.1793e-07,  1.0618e-03,  3.2822e-02,  2.7492e-03, -3.5686e-02,
        -2.7442e-02, -3.6546e-04, -1.0519e-02,  7.6228e-03, -2.9492e-02,
         2.2992e-03, -5.5533e-03,  5.0763e-03, -3.6265e-03, -5.0055e-03,
        -2.7256e

In [41]:
A = torch.tensor([2.0], requires_grad=True)

In [42]:
A.requires_grad

True

In [46]:
A.grad_fn

In [43]:
B=A**2

In [44]:
B.requires_grad

True

In [45]:
B.grad_fn